In [1]:
from models.models import Q_Model, S_Model
from datas.commondataset import ETRI_Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
import os
import pandas as pd
from tqdm import tqdm

DATA_PATH = '/workspace/jupyter_workspace/SHARED_FILES/docker_shared/'

val_path = os.path.join(DATA_PATH, 'val_dataset')
df_mLight = pd.read_parquet(os.path.join(val_path, 'ch2024_val__m_light.parquet.gzip'))
df_wHr = pd.read_parquet(os.path.join(val_path, 'ch2024_val__w_heart_rate.parquet.gzip'))
df_wLight = pd.read_parquet(os.path.join(val_path, 'ch2024_val__w_light.parquet.gzip'))

label_path = os.path.join(DATA_PATH, 'val_label.csv')


train_dataset = ETRI_Dataset(label_path, df_mLight, df_wHr, df_wLight, 's', 'noise')
train_dataloader = DataLoader(train_dataset, batch_size = 1, shuffle = False, num_workers = 4)

model = S_Model()

epochs = 100
cir = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

def acc_fn(y_pred, y_true):
    accuracy = torch.eq(y_pred, y_true).sum().item()/len(y_pred)
    return accuracy


model = model.to('cuda')

best_loss = 1

for epoch in range(epochs) :
    epoch_loss = 0
    epoch_acc = 0
    
    model = model.train()
    for i, batch in enumerate(tqdm(train_dataloader)) :
        mLight, wHr, wLight, labels = batch
        mLight, wHr, wLight = mLight.to('cuda'), wHr.to('cuda'), wLight.to('cuda')
        #print(wHr)
        #print(wLight)
        prediction = model(mLight, wHr, wLight)
        #prediction = torch.softmax(prediction, dim=1)
        loss = cir(prediction, labels[0][0].to('cuda'))
        print(prediction.argmax(dim=1))
        acc = acc_fn(prediction.argmax(dim=1), labels[0][0].to('cuda'))
        epoch_loss += loss
        epoch_acc += acc
        
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
    
    epoch_loss = epoch_loss / len(train_dataloader)
    epoch_acc = epoch_acc / len(train_dataloader)
    if best_loss > epoch_loss :
        print('save best model in epoch')
        torch.save(model.state_dict(), './best.pt') 
        best_loss = epoch_loss
    print(f"EPOCH {epoch} - Loss : {epoch_loss} , Acc : {epoch_acc}")



NameError: name 'model' is not defined